
# Copyright 2026 Google LLC

Licensed under the Apache License, Version 2.0.

# NBA Agent Trace Analytics Notebook (ADK + BigQuery Agent Analytics SDK)

This notebook demonstrates an end-to-end workflow:

```mermaid
flowchart LR
    A[Configure gcloud / gh / ADK env] --> B[Run NBA ADK agent]
    B --> C[BigQuery Agent Analytics Plugin logs events]
    C --> D[BigQuery dataset: agent_trace]
    D --> E[SDK Client reconstructs traces]
    E --> F[Code + trajectory evaluation]
    F --> G[Insights for NBA conversations]
```

Target environment defaults used in this notebook:
- **GCP project**: `your-project-id`
- **BigQuery dataset**: `agent_trace`
- **Repository**: `haiyuan-eng-google/BigQuery-Agent-Analytics-SDK`


<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/haiyuan-eng-google/BigQuery-Agent-Analytics-SDK/blob/main/examples/nba_agent_trace_analysis_notebook.ipynb">
      <img src="https://raw.githubusercontent.com/googleapis/python-bigquery-dataframes/refs/heads/main/third_party/logo/colab-logo.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/haiyuan-eng-google/BigQuery-Agent-Analytics-SDK/main/examples/nba_agent_trace_analysis_notebook.ipynb">
      <img src="https://www.gstatic.com/images/branding/product/1x/google_cloud_48dp.png" alt="Vertex AI logo" width="32"> Open in Vertex AI Workbench
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/haiyuan-eng-google/BigQuery-Agent-Analytics-SDK/blob/main/examples/nba_agent_trace_analysis_notebook.ipynb">
      <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTW1gvOovVlbZAIZylUtf5Iu8-693qS1w5NJw&s" alt="BQ logo" width="35"> Open in BQ Studio
    </a>
  </td>
</table>


## 1) Install dependencies

In [ ]:
!pip install -q google-adk bigquery-agent-analytics google-cloud-bigquery nest-asyncio pandas || pip install -q google-adk google-cloud-bigquery nest-asyncio pandas "git+https://github.com/haiyuan-eng-google/BigQuery-Agent-Analytics-SDK.git@main"


## 2) Environment setup (gcloud / gh / ADK)

In [ ]:

import os
import subprocess

PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT", "your-project-id")
DATASET_ID = "agent_trace"
TABLE_ID = "agent_events"
BQ_LOCATION = "US"
MODEL_NAME = "gemini-2.5-flash"

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["BQ_DATASET"] = DATASET_ID
os.environ["BQ_TABLE"] = TABLE_ID
os.environ.setdefault("GOOGLE_CLOUD_LOCATION", "global")
os.environ.setdefault("GOOGLE_GENAI_USE_VERTEXAI", "true")

print("Environment configured:")
for k in ["GOOGLE_CLOUD_PROJECT", "BQ_DATASET", "BQ_TABLE", "GOOGLE_CLOUD_LOCATION", "GOOGLE_GENAI_USE_VERTEXAI"]:
    print(f"  {k}={os.environ[k]}")


def _run(cmd: str):
    print(f"\n$ {cmd}")
    p = subprocess.run(cmd, shell=True, text=True, capture_output=True)
    if p.stdout:
        print(p.stdout.strip())
    if p.returncode != 0 and p.stderr:
        print(p.stderr.strip())

# Quick health checks for local tools
_run("gcloud --version")
_run("gh --version")

print("""
If auth is needed, run manually in a terminal:
  gcloud auth login
  gcloud auth application-default login
  gcloud config set project $GOOGLE_CLOUD_PROJECT
  bq --location=US mk --dataset --if_not_exists ${GOOGLE_CLOUD_PROJECT}:agent_trace
  gh auth login
""")


## 3) Define a deterministic NBA toolset for the test agent

In [ ]:

import hashlib
import random
from typing import Any


def _rng_from(*parts: str) -> random.Random:
    seed = int(hashlib.md5("|".join(parts).encode()).hexdigest()[:8], 16)
    return random.Random(seed)


async def get_nba_team_snapshot(team_name: str, season: str = "2024-25") -> dict[str, Any]:
    """Return deterministic synthetic team stats for demo trace generation."""
    rng = _rng_from(team_name, season)
    wins = rng.randint(28, 62)
    losses = 82 - wins
    pace = round(rng.uniform(95.0, 103.0), 1)
    off = round(rng.uniform(108.0, 124.0), 1)
    deff = round(rng.uniform(106.0, 121.0), 1)
    return {
        "team": team_name,
        "season": season,
        "wins": wins,
        "losses": losses,
        "net_rating": round(off - deff, 1),
        "off_rating": off,
        "def_rating": deff,
        "pace": pace,
    }


async def get_nba_player_snapshot(player_name: str, season: str = "2024-25") -> dict[str, Any]:
    """Return deterministic synthetic player box-score style metrics."""
    rng = _rng_from(player_name, season)
    return {
        "player": player_name,
        "season": season,
        "games": rng.randint(45, 82),
        "ppg": round(rng.uniform(8.0, 34.0), 1),
        "rpg": round(rng.uniform(1.5, 13.5), 1),
        "apg": round(rng.uniform(1.0, 11.5), 1),
        "ts_pct": round(rng.uniform(0.50, 0.69), 3),
    }


async def compare_matchup(home_team: str, away_team: str, season: str = "2024-25") -> dict[str, Any]:
    """Create a deterministic matchup projection and rationale."""
    home = await get_nba_team_snapshot(home_team, season)
    away = await get_nba_team_snapshot(away_team, season)
    margin = round((home["net_rating"] - away["net_rating"]) + 2.1, 1)
    favorite = home_team if margin >= 0 else away_team
    return {
        "season": season,
        "home_team": home_team,
        "away_team": away_team,
        "favorite": favorite,
        "projected_margin": abs(margin),
        "reason": (
            f"{favorite} favored due to relative net rating edge and home-court adjustment."
        ),
        "home_snapshot": home,
        "away_snapshot": away,
    }


## 4) Build ADK NBA agent with BigQuery analytics plugin

In [ ]:

import asyncio
import nest_asyncio
import time
import uuid

from google.adk.agents import LlmAgent
from google.adk.plugins.bigquery_agent_analytics_plugin import BigQueryAgentAnalyticsPlugin
from google.adk.plugins.bigquery_agent_analytics_plugin import BigQueryLoggerConfig
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

nest_asyncio.apply()

plugin = BigQueryAgentAnalyticsPlugin(
    project_id=PROJECT_ID,
    dataset_id=DATASET_ID,
    table_id=TABLE_ID,
    location=BQ_LOCATION,
    config=BigQueryLoggerConfig(),
)

NBA_AGENT_INSTRUCTION = """
You are an NBA analytics assistant.
- Always reason from available tool outputs.
- Be explicit when data is synthetic for demo purposes.
- Provide concise basketball insights in bullet points.
"""

agent = LlmAgent(
    name="nba_analytics_agent",
    model=MODEL_NAME,
    instruction=NBA_AGENT_INSTRUCTION,
    tools=[get_nba_team_snapshot, get_nba_player_snapshot, compare_matchup],
)

session_service = InMemorySessionService()
runner = Runner(
    app_name="nba_agent_trace_demo",
    agent=agent,
    session_service=session_service,
    plugins=[plugin],
)

async def run_conversation(session_id: str, user_prompt: str) -> str:
    await session_service.create_session(
        app_name="nba_agent_trace_demo",
        user_id="demo_user",
        session_id=session_id,
    )
    final_text = ""
    async for event in runner.run_async(
        user_id="demo_user",
        session_id=session_id,
        new_message=types.Content(role="user", parts=[types.Part(text=user_prompt)]),
    ):
        if event.is_final_response() and event.content and event.content.parts:
            final_text = event.content.parts[0].text or ""
    return final_text

prompts = [
    "Compare Lakers vs Celtics in a hypothetical Finals matchup and explain key edges.",
    "Give me a scouting snapshot for Nikola Jokic, then compare Nuggets vs Bucks.",
    "Who has the better 2024-25 profile: Knicks or 76ers? Include pace and ratings.",
    "Break down Warriors vs Suns: expected favorite, projected margin, and why.",
    "Create quick team snapshots for Heat and Cavaliers, then recommend who has a better playoff profile.",
]

session_ids = []
for prompt in prompts:
    sid = f"nba-{uuid.uuid4().hex[:10]}"
    session_ids.append(sid)
    print(f"\n=== Running session {sid} ===")
    print("User:", prompt)
    response = asyncio.run(run_conversation(sid, prompt))
    print("Agent:", response[:600], "..." if len(response) > 600 else "")

# Give plugin time to flush in notebook environments.
time.sleep(3)
print("\nTrace sessions:", session_ids)

# Wait/poll until plugin writes become visible in BigQuery.
from google.cloud import bigquery

bq_client = bigquery.Client(project=PROJECT_ID, location=BQ_LOCATION)
max_wait_s = 120
poll_s = 10
elapsed = 0
rows_visible = 0

while elapsed <= max_wait_s:
    query = f"""
    SELECT COUNT(*) AS c
    FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
    WHERE session_id IN UNNEST(@session_ids)
    """
    job = bq_client.query(
        query,
        job_config=bigquery.QueryJobConfig(
            query_parameters=[
                bigquery.ArrayQueryParameter("session_ids", "STRING", session_ids)
            ]
        ),
    )
    rows_visible = int(list(job.result())[0]["c"])
    print(f"Elapsed {elapsed:>3}s -> visible_rows={rows_visible}")
    if rows_visible > 0:
        break
    time.sleep(poll_s)
    elapsed += poll_s

print("Final visible rows for current notebook sessions:", rows_visible)


## 5) Analyze traces with BigQuery Agent Analytics SDK APIs


In [ ]:

from bigquery_agent_analytics import BigQueryTraceEvaluator
from bigquery_agent_analytics import Client
from bigquery_agent_analytics import CodeEvaluator
from bigquery_agent_analytics.trace_evaluator import MatchType

client = Client(
    project_id=PROJECT_ID,
    dataset_id=DATASET_ID,
    table_id=TABLE_ID,
    location=BQ_LOCATION,
)

target_session = session_ids[0]
traces = client.list_traces()
trace = next(t for t in traces if t.session_id == target_session)
print("Loaded trace for session:", target_session, "trace_id:", trace.trace_id)

# In notebooks, Trace.render() returns a graphviz object if available.
rendered = trace.render()
rendered


In [ ]:
# Quick trace table instead of raw SQL
trace_rows = []
for t in client.list_traces():
    trace_rows.append({
        "session_id": t.session_id,
        "trace_id": t.trace_id,
        "user_id": t.user_id,
        "span_count": len(t.spans),
        "total_latency_ms": t.total_latency_ms,
    })

import pandas as pd
pd.DataFrame(trace_rows).head(10)


In [ ]:
# Deterministic code-based evaluation examples (SDK client-side)
latency_eval = CodeEvaluator.latency(threshold_ms=12000)
turn_eval = CodeEvaluator.turn_count(max_turns=16)

report_latency = client.evaluate(latency_eval)
report_turns = client.evaluate(turn_eval)
print("latency pass_rate:", report_latency.pass_rate)
print("turn_count pass_rate:", report_turns.pass_rate)


### 5.1) LLM-as-Judge example (factuality / tactical depth proxy)
This uses the SDK `LLMAsJudge` path over the same generated NBA sessions.


In [ ]:
from bigquery_agent_analytics import LLMAsJudge, TraceFilter

judge = LLMAsJudge.correctness(threshold=0.6, model=MODEL_NAME)
judge_report = client.evaluate(
    judge,
    filters=TraceFilter(session_ids=session_ids),
)

print("judge evaluator:", judge_report.evaluator_name)
print("sessions evaluated:", judge_report.total_sessions)
print("pass_rate:", judge_report.pass_rate)
judge_report.session_scores[:3]


In [ ]:

# Trajectory matching against expected tool behavior
trajectory_evaluator = BigQueryTraceEvaluator(
    project_id=PROJECT_ID,
    dataset_id=DATASET_ID,
    table_id=TABLE_ID,
)

expected = [
    {"tool_name": "get_nba_team_snapshot"},
    {"tool_name": "compare_matchup"},
]

traj_result = asyncio.run(
    trajectory_evaluator.evaluate_session(
        session_id=target_session,
        golden_trajectory=expected,
        match_type=MatchType.ANY_ORDER,
    )
)

print("Eval status:", traj_result.eval_status)
print("Overall score:", traj_result.overall_score)
print("Scores:", traj_result.scores)
print("Details:", traj_result.details)


## 6) Optional: Generate insights report for NBA sessions

In [ ]:

from bigquery_agent_analytics import InsightsConfig
from bigquery_agent_analytics import TraceFilter

config = InsightsConfig(max_sessions=len(session_ids))

insights = client.insights(
    config=config,
    filters=TraceFilter(session_ids=session_ids),
)
insights



## 7) Next steps

- Swap synthetic tool outputs with a real NBA data source (e.g., trusted sports API).
- Track model versions in metadata to compare trace quality over time.
